In [ ]:
!wget https://github.com/rinstance/sign_language_scanner_update/archive/master.zip -O master.zip

In [ ]:
!unzip master.zip

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import tensorflow as tf

im_height = 224
im_width = 224
batch_size = 32
epochs = 50

train_dir = 'sign_language_scanner_update-master/classes'
validation_dir = 'sign_language_scanner_update-master/classes'

train_image_generator = ImageDataGenerator( rescale=1./255,
                                            rotation_range=40,
                                            width_shift_range=0.2,
                                            height_shift_range=0.2,
                                            shear_range=0.2,
                                            zoom_range=0.2,
                                            horizontal_flip=True,
                                            fill_mode='nearest')

train_data_gen = train_image_generator.flow_from_directory(directory=train_dir,
                                                           batch_size=batch_size,
                                                           shuffle=True,
                                                           target_size=(im_height, im_width),
                                                           class_mode='categorical')
total_train = train_data_gen.n

validation_image_generator = ImageDataGenerator(rescale=1./255)

val_data_gen = validation_image_generator.flow_from_directory(directory=validation_dir,
                                                              batch_size=batch_size,
                                                              shuffle=False,
                                                              target_size=(im_height, im_width),
                                                              class_mode='categorical')
total_val = val_data_gen.n

covn_base = tf.keras.applications.MobileNetV2(weights='imagenet',include_top=False)
covn_base.trainable = True

for layers in covn_base.layers[:-5]:
    layers.trainable = False

model = tf.keras.Sequential()
model.add(covn_base)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(29,activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=["accuracy"])
model.summary()

history = model.fit(x=train_data_gen,
                    steps_per_epoch=total_train,
                    epochs=epochs,
                    validation_data=val_data_gen,
                    validation_steps=total_val)

model.save("model.h5")

In [ ]:
import pathlib

TF_LITE_MODEL_FILE_NAME = 'model_sign_detection.tflite'

tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = tf_lite_converter.convert()

tflite_model_file = pathlib.Path(TF_LITE_MODEL_FILE_NAME)
tflite_model_file.write_bytes(tflite_model)